## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ayan,RU,56.4500,138.1667,59.05,89,70,8.59,broken clouds
1,1,Polignano A Mare,IT,40.9959,17.2159,77.72,75,0,13.73,clear sky
2,2,Saint-Philippe,RE,-21.3585,55.7679,71.11,77,42,13.76,scattered clouds
3,3,Shetpe,KZ,44.1667,52.1167,76.37,67,0,5.59,clear sky
4,4,East London,ZA,-33.0153,27.9116,57.24,63,19,14.29,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you want for your trip? "))

What is the minimum temperature you want for your trip? 70.0


In [4]:
max_temp = float(input("What is the maximum temperature you want for your trip? "))

What is the maximum temperature you want for your trip? 80.0


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Polignano A Mare,IT,40.9959,17.2159,77.72,75,0,13.73,clear sky
2,2,Saint-Philippe,RE,-21.3585,55.7679,71.11,77,42,13.76,scattered clouds
3,3,Shetpe,KZ,44.1667,52.1167,76.37,67,0,5.59,clear sky
11,11,Hamilton,US,39.1834,-84.5333,76.05,66,0,5.75,clear sky
20,20,Saint-Joseph,RE,-21.3667,55.6167,70.11,76,37,13.13,scattered clouds
21,21,Harper,LR,4.3750,-7.7169,75.07,81,100,7.85,overcast clouds
22,22,Albany,US,42.6001,-73.9662,74.05,63,0,8.39,clear sky
30,30,Hobyo,SO,5.3505,48.5268,74.95,85,83,24.14,broken clouds
33,33,Welwyn Garden City,GB,51.8017,-0.2069,70.25,82,40,11.50,scattered clouds
35,35,Alyangula,AU,-13.8483,136.4192,76.93,65,5,5.39,clear sky


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                213
City                   213
Country                213
Lat                    213
Lng                    213
Max Temp               213
Humidity               213
Cloudiness             213
Wind Speed             213
Current Description    213
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df= preferred_cities_df.dropna()
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Polignano A Mare,IT,40.9959,17.2159,77.72,75,0,13.73,clear sky
2,2,Saint-Philippe,RE,-21.3585,55.7679,71.11,77,42,13.76,scattered clouds
3,3,Shetpe,KZ,44.1667,52.1167,76.37,67,0,5.59,clear sky
11,11,Hamilton,US,39.1834,-84.5333,76.05,66,0,5.75,clear sky
20,20,Saint-Joseph,RE,-21.3667,55.6167,70.11,76,37,13.13,scattered clouds


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Polignano A Mare,IT,77.72,clear sky,40.9959,17.2159,
2,Saint-Philippe,RE,71.11,scattered clouds,-21.3585,55.7679,
3,Shetpe,KZ,76.37,clear sky,44.1667,52.1167,
11,Hamilton,US,76.05,clear sky,39.1834,-84.5333,
20,Saint-Joseph,RE,70.11,scattered clouds,-21.3667,55.6167,
21,Harper,LR,75.07,overcast clouds,4.3750,-7.7169,
22,Albany,US,74.05,clear sky,42.6001,-73.9662,
30,Hobyo,SO,74.95,broken clouds,5.3505,48.5268,
33,Welwyn Garden City,GB,70.25,scattered clouds,51.8017,-0.2069,
35,Alyangula,AU,76.93,clear sky,-13.8483,136.4192,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
# 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

City                   213
Country                213
Max Temp               213
Current Description    213
Lat                    213
Lng                    213
Hotel Name             213
dtype: int64

In [11]:
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Polignano A Mare,IT,77.72,clear sky,40.9959,17.2159,Grotta Palazzese
2,Saint-Philippe,RE,71.11,scattered clouds,-21.3585,55.7679,Le four à pain
3,Shetpe,KZ,76.37,clear sky,44.1667,52.1167,Rtsm Shetpe Analog
11,Hamilton,US,76.05,clear sky,39.1834,-84.5333,Six Acres Bed & Breakfast
20,Saint-Joseph,RE,70.11,scattered clouds,-21.3667,55.6167,"""Plantation Bed and Breakfast"
21,Harper,LR,75.07,overcast clouds,4.3750,-7.7169,Screensaver
22,Albany,US,74.05,clear sky,42.6001,-73.9662,
30,Hobyo,SO,74.95,broken clouds,5.3505,48.5268,Iftin hotel hobyo
33,Welwyn Garden City,GB,70.25,scattered clouds,51.8017,-0.2069,Premier Inn Welwyn Garden City hotel
35,Alyangula,AU,76.93,clear sky,-13.8483,136.4192,Groote Eylandt Lodge


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))